# Finetuning the last year of fine-rune model

In [1]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


#importing fundtion to get encoder info and perfrom tasks 
from finetune.get_finetune_encoder import  get_finetune_input_data

from models.models_VAE import VAE

%who



[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
2024-09-18 22:12:29.860920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 22:12:29.883702: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 22:12:29.890826: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 22:12:29.908686: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions

DataLoader	 LabelEncoder	 TensorDataset	 VAE	 accuracy_score	 f1_score	 get_finetune_input_data	 imaplib	 importlib	 
label_encoder	 ml_code_path	 nn	 np	 optim	 optuna	 os	 pd	 precision_score	 
random	 recall_score	 roc_auc_score	 torch	 


## gettin the model ID and info

#### getting the input data

In [8]:
#input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
finetune_save_dir='/home/leilapirhaji/finetune_VAE_NO_RCC' 

#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

#survival tasks
task_list_survival=[ 'OS', 'NIVO OS', 'EVER OS', 'PFS']


#get fine-tuning input data 
(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_finetune_input_data(input_data_location)

X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((443, 2736), (443, 58), (149, 2736), (149, 58), (149, 2736), (149, 58))

### loading the fine-tune VAE models w/wo transfer leanrning

In [9]:
#first getting A list of pretrained models and their associated fine-tune 
#pretrain_model_list=os.listdir(finetune_save_dir)
#pretrain_model_list=['RCC-38529']
pretrain_model_ID='RCC-38773'

In [10]:
from finetune.retrain_finetune_VAE_COX import get_finetune_VAE_TL_noTL

(finetune_VAE_TL_OG, finetune_VAE_noTL_OG)=get_finetune_VAE_TL_noTL(pretrain_model_ID, finetune_save_dir )

/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_finetune_VAE_COX.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  finetune_VAE_TL=torch.load(finetune_VAE_TL_file)

In [11]:
finetune_VAE_TL_OG.latent_size, finetune_VAE_TL_OG.num_hidden_layers, finetune_VAE_noTL_OG.latent_size, finetune_VAE_noTL_OG.num_hidden_layers

(360, 2, 360, 2)

## Visualizing the latent space

In [ ]:
import matplotlib.pyplot as plt
import finetune.eval_finetune_latent_main

importlib.reload(finetune.eval_finetune_latent_main)
from finetune.eval_finetune_latent_main import visualize_latent_space_multiple_tasks

models_path=f'{finetune_save_dir}/{pretrain_model_ID}'
result_png_file_TL= f'{models_path}/finetune_VAE_{pretrain_model_ID}_TL_latent_space.png'

visualize_latent_space_multiple_tasks(finetune_VAE_TL, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, result_png_file_TL)

plt.show()


## Perdciting tasks from latnet space

In [12]:
#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

task= 'IMDC BINARY'


import finetune.eval_finetune_latent_main

importlib.reload(finetune.eval_finetune_latent_main)
from finetune.eval_finetune_latent_main import predict_task_from_latent_avg



best_val_accuracy, best_val_auc, test_accuracy, test_auc= predict_task_from_latent_avg (finetune_VAE_TL_OG, task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

best_val_accuracy, best_val_auc, test_accuracy, test_auc

Test Accuracy with best model: 0.7895
Test AUC with best model: 0.8760


(0.8360655737704918, 0.87995337995338, 0.7894736842105263, 0.8759590792838874)

In [13]:
best_val_accuracy, best_val_auc, test_accuracy, test_auc= predict_task_from_latent_avg (finetune_VAE_noTL_OG, task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

test_accuracy, test_auc

Test Accuracy with best model: 0.7544
Test AUC with best model: 0.8645


(0.7543859649122807, 0.8644501278772379)

## Predicting Survival from latnent avergae

### OS prediction

In [25]:
import finetune.eval_finetune_latent_main
importlib.reload(finetune.eval_finetune_latent_main)
from finetune.eval_finetune_latent_main import predict_survival_from_latent_avg


task= 'NIVO OS'
task_event='OS_Event'

best_val_c_index, best_test_c_index, best_params= predict_survival_from_latent_avg (finetune_VAE_TL_OG, task, task_event, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning:

all coefficients are zero, consider decreasing alpha.

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning:

all coefficients are zero, consider decreasing alpha.



(0.5983063328424153, 0.6214233308877476, 0.01)

In [26]:
best_val_c_index, best_test_c_index, best_params= predict_survival_from_latent_avg (finetune_VAE_noTL_OG, task, task_event, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning:

all coefficients are zero, consider decreasing alpha.

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning:

all coefficients are zero, consider decreasing alpha.

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning:

all coefficients are zero, consider decreasing alpha.



(0.6689985272459499, 0.66507703595011, 0.01)

## VAE model with fine-tuned last layer

In [14]:
#loading the seed file
with open('/home/leilapirhaji/mz_embed_engine/ml/config/seed_Aug31.txt', 'r') as f:
    seed = int(f.read())

In [52]:
import copy
from finetune.retrain_finetune_VAE import retrain_finetune_VAE_TL_noTL_fixed_hyper_par

# Create a deep copy of the model before retraining
finetune_VAE_TL = copy.deepcopy(finetune_VAE_TL_OG)
finetune_VAE_noTL = copy.deepcopy(finetune_VAE_noTL_OG)

task= 'IMDC BINARY'
num_classes=2


results_combined_df= retrain_finetune_VAE_TL_noTL_fixed_hyper_par(finetune_VAE_TL, finetune_VAE_noTL, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, task, num_classes, seed, num_layers_to_retrain=2, add_post_latent_layers=False, post_latent_layer_size=128, num_epochs=10, learning_rate=1e-5, dropout=0.2, l1_reg_weight= 1e-5, l2_reg_weight= 1e-7, latent_passes= 20)

Epoch 1/10, Loss: 0.7790328647409167
Validation started
Validation Loss: 0.764324676990509, Accuracy: 60.65573770491803%, Precision: 45.45454545454545%, Recall: 45.45454545454545%, F1 Score: 45.45454545454545%, AUC: 64.45221445221445%


/home/leilapirhaji/mz_embed_engine/ml/finetune/freez_encoder_latent_avg.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_per_epoch = pd.concat([metrics_per_epoch, metrics_df], ignore_index=True)


Epoch 2/10, Loss: 0.75808556165014
Validation started
Validation Loss: 0.737964391708374, Accuracy: 68.85245901639344%, Precision: 60.0%, Recall: 40.909090909090914%, F1 Score: 48.64864864864865%, AUC: 75.40792540792542%
Epoch 3/10, Loss: 0.7422085319246564
Validation started
Validation Loss: 0.7149431228637695, Accuracy: 70.49180327868852%, Precision: 64.28571428571429%, Recall: 40.909090909090914%, F1 Score: 50.0%, AUC: 79.60372960372962%
Epoch 4/10, Loss: 0.7248204180172512
Validation started
Validation Loss: 0.6928697109222413, Accuracy: 73.77049180327869%, Precision: 66.66666666666666%, Recall: 54.54545454545454%, F1 Score: 60.0%, AUC: 82.86713286713288%
Epoch 5/10, Loss: 0.7046399669987815
Validation started
Validation Loss: 0.6773138523101807, Accuracy: 75.40983606557377%, Precision: 66.66666666666666%, Recall: 63.63636363636363%, F1 Score: 65.11627906976744%, AUC: 84.26573426573427%
Epoch 6/10, Loss: 0.685689892087664
Validation started
Validation Loss: 0.6568703413009643, Accu

/home/leilapirhaji/mz_embed_engine/ml/finetune/freez_encoder_latent_avg.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_per_epoch = pd.concat([metrics_per_epoch, metrics_df], ignore_index=True)


Epoch 2/10, Loss: 0.8639052765710014
Validation started
Validation Loss: 0.8509699702262878, Accuracy: 63.934426229508205%, Precision: 50.0%, Recall: 45.45454545454545%, F1 Score: 47.61904761904761%, AUC: 69.58041958041959%
Epoch 3/10, Loss: 0.8503130120890481
Validation started
Validation Loss: 0.8341515779495239, Accuracy: 68.85245901639344%, Precision: 57.14285714285714%, Recall: 54.54545454545454%, F1 Score: 55.81395348837209%, AUC: 77.85547785547786%
Epoch 4/10, Loss: 0.8357714499746051
Validation started
Validation Loss: 0.8159685730934143, Accuracy: 77.04918032786885%, Precision: 65.38461538461539%, Recall: 77.27272727272727%, F1 Score: 70.83333333333334%, AUC: 82.75058275058275%
Epoch 5/10, Loss: 0.8194145389965602
Validation started
Validation Loss: 0.8000946998596191, Accuracy: 77.04918032786885%, Precision: 65.38461538461539%, Recall: 77.27272727272727%, F1 Score: 70.83333333333334%, AUC: 85.08158508158508%
Epoch 6/10, Loss: 0.8023774921894073
Validation started
Validation L

In [53]:
results_combined_df

,Model,Dataset,Accuracy,Precision,Recall,F1 Score,AUC,Test Loss
0,Transfer Learning,Train,78.709677,86.000000,62.318841,72.268908,91.085271,0.0
1,Transfer Learning,Validation,83.606557,75.000000,81.818182,78.260870,86.829837,0.0
2,Transfer Learning,Test,78.947368,73.913043,73.913043,73.913043,91.048593,0.0
3,No Transfer Learning,Train,78.709677,75.714286,76.811594,76.258993,87.226154,0.0
4,No Transfer Learning,Validation,81.967213,70.370370,86.363636,77.551020,88.578089,0.0
5,No Transfer Learning,Test,77.192982,69.230769,78.260870,73.469388,83.887468,0.0


## Re-training the encoder to predict classification tasks 

In [29]:
#makign seeds that are the same for all the tasks
#import set_seed
#from set_seed import set_seed
#seed = set_seed(42)  # Explicitly set and save the seed

#with open('/home/leilapirhaji/mz_embed_engine/ml/seed_Aug31.txt', 'w') as f:
#    f.write(str(seed))
#loading the seed file
with open('/home/leilapirhaji/mz_embed_engine/ml/config/seed_Aug31.txt', 'r') as f:
    seed = int(f.read())

### optuna optimization -  TL optimiation

In [39]:
import finetune.retrain_finetune_VAE
importlib.reload(finetune.retrain_finetune_VAE)
from finetune.retrain_finetune_VAE import retrain_finetune_VAE_TL_noTL_Optuna_optimization

task='MSKCC BINARY'
num_classes =2

study, results_combined_df= retrain_finetune_VAE_TL_noTL_Optuna_optimization(finetune_VAE_TL, finetune_VAE_noTL, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, task, num_classes, seed, finetune_save_dir, pretrain_model_ID, n_trials=40)

results_combined_df

[I 2024-09-17 03:18:06,670] A new study created in memory with name: no-name-cf961eae-4f4c-4306-a992-54aaa267a966
/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_finetune_VAE.py:143: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_finetune_VAE.py:144: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_finetune_VAE.py:145: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_

Epoch 1/50, Loss: 0.755298376083374
Validation started
Validation Loss: 0.7864187121391296, Accuracy: 65.85365853658537%, Precision: 65.85365853658537%, Recall: 100.0%, F1 Score: 79.41176470588235%, AUC: 87.89682539682539%
Epoch 2/50, Loss: 0.7409061023167202
Validation started


/home/leilapirhaji/mz_embed_engine/ml/finetune/freez_encoder_latent_avg.py:370: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Validation Loss: 0.7815972328186035, Accuracy: 65.85365853658537%, Precision: 65.85365853658537%, Recall: 100.0%, F1 Score: 79.41176470588235%, AUC: 88.02910052910053%
Epoch 3/50, Loss: 0.7336526683398655
Validation started
Validation Loss: 0.7768700122833252, Accuracy: 65.85365853658537%, Precision: 65.85365853658537%, Recall: 100.0%, F1 Score: 79.41176470588235%, AUC: 88.16137566137566%
Epoch 4/50, Loss: 0.7324268519878387
Validation started
Validation Loss: 0.7723245739936828, Accuracy: 65.85365853658537%, Precision: 65.85365853658537%, Recall: 100.0%, F1 Score: 79.41176470588235%, AUC: 88.29365079365078%
Epoch 5/50, Loss: 0.7314274864537376
Validation started
Validation Loss: 0.7677964329719543, Accuracy: 70.73170731707317%, Precision: 69.73684210526315%, Recall: 98.14814814814815%, F1 Score: 81.53846153846153%, AUC: 88.29365079365078%
Epoch 6/50, Loss: 0.7066515982151031
Validation started
Validation Loss: 0.7636435389518738, Accuracy: 73.17073170731707%, Precision: 71.62162162162

/home/leilapirhaji/mz_embed_engine/ml/finetune/freez_encoder_latent_avg.py:370: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



...

Epoch 2/50, Loss: 2.459902831486293
Validation started
Validation Loss: 3.3107625007629395, Accuracy: 78.04878048780488%, Precision: 75.71428571428571%, Recall: 98.14814814814815%, F1 Score: 85.48387096774194%, AUC: 84.12698412698413%
Epoch 3/50, Loss: 2.4472701379231045
Validation started
Validation Loss: 3.2784179210662843, Accuracy: 80.48780487804879%, Precision: 79.6875%, Recall: 94.44444444444444%, F1 Score: 86.4406779661017%, AUC: 84.39153439153439%
Epoch 4/50, Loss: 2.4399146863392422
Validation started
Validation Loss: 3.2555453777313232, Accuracy: 81.70731707317073%, Precision: 80.95238095238095%, Recall: 94.44444444444444%, F1 Score: 87.17948717948718%, AUC: 84.39153439153439%
Epoch 5/50, Loss: 2.4357649428503856
Validation started
Validation Loss: 3.239334487915039, Accuracy: 81.70731707317073%, Precision: 80.95238095238095%, Recall: 94.44444444444444%, F1 Score: 87.17948717948718%, AUC: 84.52380952380952%
Epoch 6/50, Loss: 2.430988686425345
Validation started
Validation Los

[I 2024-09-17 03:20:47,204] Trial 39 finished with value: 88.55820105820106 and parameters: {'add_post_latent_layers': False, 'num_layers_to_retrain': 1, 'batch_size': 32, 'learning_rate': 1.4603483758045745e-05, 'dropout': 0.25455756889688774, 'l1_reg_weight': 0.00015464406447363924, 'l2_reg_weight': 1.883849912354062e-06}. Best is trial 18 with value: 88.82275132275133.


Validation Loss: 0.0, Accuracy: 100.0%, Precision: 100.0%, Recall: 100.0%, F1 Score: 100.0%, AUC: 100.0%
Fine-tuning completed.
Validation Loss: 0.0, Accuracy: 79.26829268292683%, Precision: 81.35593220338984%, Recall: 88.88888888888889%, F1 Score: 84.95575221238938%, AUC: 88.55820105820106%
Fine-tuning completed.
Validation Loss: 0.0, Accuracy: 83.33333333333334%, Precision: 81.9672131147541%, Recall: 94.33962264150944%, F1 Score: 87.71929824561403%, AUC: 88.67924528301887%
Fine-tuning completed.
Validation Loss: 0.0, Accuracy: 100.0%, Precision: 100.0%, Recall: 100.0%, F1 Score: 100.0%, AUC: 100.0%
Fine-tuning completed.
Validation Loss: 0.0, Accuracy: 79.26829268292683%, Precision: 80.32786885245902%, Recall: 90.74074074074075%, F1 Score: 85.21739130434783%, AUC: 84.65608465608466%
Fine-tuning completed.
Validation Loss: 0.0, Accuracy: 82.14285714285714%, Precision: 80.64516129032258%, Recall: 94.33962264150944%, F1 Score: 86.95652173913044%, AUC: 90.9312233718807%
Fine-tuning compl

/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_finetune_VAE.py:216: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/home/leilapirhaji/mz_embed_engine/ml/finetune/retr

,Model,Dataset,Accuracy,Precision,Recall,F1 Score,AUC,Test Loss
0,Transfer Learning,Train,100.000000,100.000000,100.000000,100.000000,100.000000,0.0
1,Transfer Learning,Validation,79.268293,81.355932,88.888889,84.955752,88.558201,0.0
2,Transfer Learning,Test,83.333333,81.967213,94.339623,87.719298,88.679245,0.0
3,No Transfer Learning,Train,100.000000,100.000000,100.000000,100.000000,100.000000,0.0
4,No Transfer Learning,Validation,79.268293,80.327869,90.740741,85.217391,84.656085,0.0
5,No Transfer Learning,Test,82.142857,80.645161,94.339623,86.956522,90.931223,0.0


In [40]:
results_combined_df

,Model,Dataset,Accuracy,Precision,Recall,F1 Score,AUC,Test Loss
0,Transfer Learning,Train,100.000000,100.000000,100.000000,100.000000,100.000000,0.0
1,Transfer Learning,Validation,79.268293,81.355932,88.888889,84.955752,88.558201,0.0
2,Transfer Learning,Test,83.333333,81.967213,94.339623,87.719298,88.679245,0.0
3,No Transfer Learning,Train,100.000000,100.000000,100.000000,100.000000,100.000000,0.0
4,No Transfer Learning,Validation,79.268293,80.327869,90.740741,85.217391,84.656085,0.0
5,No Transfer Learning,Test,82.142857,80.645161,94.339623,86.956522,90.931223,0.0


In [41]:
try:
    import optuna.visualization as vis
    vis.plot_optimization_history(study).show()
    vis.plot_param_importances(study).show()
except ImportError:
    print("Optuna visualization is not available. Install optuna[visualization] for plotting.")


In [42]:
study.best_trial.value, study.best_trial.params

(88.82275132275133,
 {'add_post_latent_layers': False,
  'num_layers_to_retrain': 1,
  'batch_size': 32,
  'learning_rate': 2.3869610916227712e-05,
  'dropout': 0.3292480543156034,
  'l1_reg_weight': 4.075678609967762e-06,
  'l2_reg_weight': 3.634635518387705e-07})

### Plotting the latnet space

In [ ]:
import matplotlib.pyplot as plt

import best_finetune_model_test_eval
importlib.reload(best_finetune_model_test_eval)
from best_finetune_model_test_eval import best_model_latent_plot_combined

fig, axes= best_model_latent_plot_combined(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, task)

plt.show()